In [1]:
import scipy.io as sio
import numpy as np
import pandas as pd

In [2]:
variable = sio.loadmat("./kf_practical/data/sensor_data.mat")

In [3]:
for k, v in variable.items():
    print(k)

__header__
__version__
__globals__
data
variableNames


In [4]:
print(variable["variableNames"])
print(len(variable["variableNames"][0]))
print(len(variable["data"][0]))

[[array(['Timestamp [s]'], dtype='<U13')
  array(['Global X [m]'], dtype='<U12')
  array(['Global Y [m]'], dtype='<U12') array(['Range [m]'], dtype='<U9')
  array(['Bearing [rad]'], dtype='<U13')]]
5
5


In [5]:
column_name = [i[0].split(" ")[0] for i in variable["variableNames"][0]]
column_name[1] += " X"
column_name[2] += " Y"
column_name

['Timestamp', 'Global X', 'Global Y', 'Range', 'Bearing']

In [6]:
raw_data = np.vstack([i for i in variable["data"]]);
print(raw_data.shape)

(1031, 5)


In [7]:
sensor_data = pd.DataFrame(columns=column_name, data=raw_data)
sensor_data.head(5)

,Timestamp,Global X,Global Y,Range,Bearing
0,0.0,179.820254,276.504102,148.715430,1.008220
1,0.1,179.684454,275.593219,148.637634,1.004353
2,0.2,179.947281,274.939092,148.496555,1.003498
3,0.3,179.805348,274.329455,149.586602,1.000930
4,0.4,180.460459,273.344521,148.655644,0.995270


In [8]:
variable = sio.loadmat("./kf_practical/data/groundtruth.mat")
for k, v in variable.items():
    print(k)

__header__
__version__
__globals__
gt
variableNames


In [9]:
column_name = [i[0].split(" ")[0] for i in variable["variableNames"][0]]
column_name[1] += " X"
column_name[2] += " Y"
column_name

['Timestamp', 'Global X', 'Global Y', 'Range', 'Bearing']

In [10]:
raw_data = np.vstack([i for i in variable["gt"]]);
print(raw_data.shape)

(1031, 5)


In [11]:
groundTruth = pd.DataFrame(data=raw_data, columns=column_name)
groundTruth.head(5)

,Timestamp,Global X,Global Y,Range,Bearing
0,0.0,179.807477,276.261422,149.369274,1.007121
1,0.1,179.898912,275.593810,148.854430,1.004206
2,0.2,179.990348,274.907654,148.325243,1.001202
3,0.3,180.081783,274.221497,147.797403,0.998178
4,0.4,180.173218,273.535341,147.270925,0.995131


# Linear Kalman Filter

In [12]:
sigma_X = sensor_data["Global X"].var()
sigma_Y = sensor_data["Global Y"].var() 

In [14]:
dt = 0.1
# State transition matrix
F = np.array([[1, dt, 0, 0],
              [0,  1, 0, 0],
              [0, 0, 1, dt],
              [0, 0, 0,  1]])
# measurement matrix
H = np.array([[1, 0, 0, 0],
              [0, 0, 1, 0]])
x = np.zeros((4, len(raw_data)))

In [15]:
R = np.array([[sigma_X**2, 0],
              [0, sigma_Y**2]])

In [31]:
vx = np.gradient(sensor_data["Global X"].to_numpy())
vy = np.gradient(sensor_data["Global Y"].to_numpy())
ax = np.gradient(vx)
ay = np.gradient(vy)

In [32]:
ax_m = np.mean(ax)
ay_m = np.mean(ay)
print(ax_m)
print(ay_m)

0.0013995084137268556
-1.5187692280122468e-05


In [33]:
ds_x = 0.5 * ax_m * dt ** 2
ds_y = 0.5 * ay_m * dt ** 2
dv_x = ax_m * dt
dv_y = ay_m * dt

In [34]:
Q = np.array([[1, 0],
              [0, 1]])

In [35]:
P = np.array([[100, 0],
              [0, 100]])

In [ ]:
for i in range(1, len(raw_data)):
    xa = np.array([[sensor_data["Global X"][i],
                   [sensor_data["Global Y"][i]]]])
    Pa = F.dot(P).dot(F.T) + Q
    
    dz = sensor_